In [15]:
### basic libraries
import os
import itertools
from datetime import datetime
from json import dumps
import pandas as pd 
import numpy as np

### carto libraries
import shapefile ### not necessary ?
import geopandas as gp
#from shapely.geometry import Polygon
from shapely.geometry import shape, Point

### for plotting in Jupyter
%matplotlib inline
import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams["figure.figsize"] = (10.0, 10.0)
#plt.style.use("bmh")
plt.style.use("ggplot")

from app.scripts.topojson import topojson

from pyproj import Proj, transform ### not necessary ?

crs_lambert93 = 2154
crs_WSG84     = 4326
inCRS  = 'epsg:%s' %(crs_lambert93)
outCRS = 'epsg:%s' %(crs_WSG84)
inProj  = Proj(init=inCRS)  # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84

### TO DO : find topojson libraries to reduce output file size


In [16]:
### basic folders addresses and names

cwd = os.getcwd()

data_folder      = "app/static/data"

stats_folder     = "stats"
_web             = "_web" 
stats_web_folder = stats_folder + _web

stats_path     = os.path.join(cwd, data_folder, stats_folder)
stats_web_path = os.path.join(cwd, data_folder, stats_web_folder)


carto_folder     = "carto"
_web             = "_web"
carto_web_folder = carto_folder + _web

carto_path     = os.path.join(cwd, data_folder, carto_folder)
carto_web_path = os.path.join(cwd, data_folder, carto_web_folder)



print "-- cwd :"            , cwd
print "-- stats_web_path : ", stats_web_path
print "-- carto_web_path : ", carto_web_path

-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- stats_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web
-- carto_web_path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto_web


In [17]:
### read stations_web.csv from : data/stats_web

_sep_csv         = ";"
csv_encoding_web = "utf-8"

stations_filename = "stations_web.csv"
stations_web_filepath = os.path.join(stats_web_path, stations_filename)
print stations_web_filepath

df_stations_web = pd.read_csv( stations_web_filepath, sep=_sep_csv, encoding=csv_encoding_web )

df_stations_web.head(3)

/Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/stats_web/stations_web.csv


,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,reseau2009,...,fi_ma_2007,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,LAT_WSG84,LONG_WSG84
0,NaN,NaN,NaN,NaN,10261X0039/F3,NaN,AEAG,10.0,33,Hors RCS et RCO,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856
1,NaN,NaN,NaN,CG004,01688X0034/AVAL,NaN,AERM,235.0,NaN,Hors RCS et RCO,...,NaN,NaN,oui,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856
2,NaN,NaN,NaN,CG004,01688X0039/F1,NaN,AERM,170.0,80,Hors RCS et RCO,...,oui,NaN,oui,NaN,NaN,NaN,NaN,NaN,-1.363081,-5.983856


In [18]:
### create point geometries

geometry     = [ Point(xy) for xy in zip(df_stations_web.LAT_WSG84, df_stations_web.LONG_WSG84) ]

gdf_stations = gp.GeoDataFrame(df_stations_web, crs=crs_WSG84, geometry=geometry)

gdf_stations.tail(3)

,NUM_DEP,NOM_COM,CD_ME_niv1_surf,CD_ME_v2,CD_STATION,NUM_COM,codagence,ALTITUDE,PROFONDEUR_MAXI_POINT,reseau2009,...,fi_ma_2008,fi_ma_2009,fi_ma_2010,fi_ma_2011,fi_ma_2012,fi_ma_2013,fi_ma_2014,LAT_WSG84,LONG_WSG84,geometry
13036,95,VIENNE-EN-ARTHIES,HG107,NaN,01514X0029/P,95656,AESN,100.0,6.5,Hors RCS et RCO,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,1.721380,49.052672,POINT (1.72137996602 49.0526718809)
13037,95,VILLERS-EN-ARTHIES,HG107,NaN,01514X0005/HY,95676,AESN,100.0,NaN,Hors RCS et RCO,...,NaN,oui,NaN,NaN,NaN,NaN,NaN,1.744907,49.090419,POINT (1.74490703083 49.0904192425)
13038,95,WY-DIT-JOLI-VILLAGE,HG107,NaN,01521X0004/HY,95690,AESN,110.0,NaN,Hors RCS et RCO,...,oui,NaN,NaN,NaN,NaN,NaN,NaN,1.838831,49.098160,POINT (1.83883087182 49.0981601538)


In [19]:
#import fiona
#fiona.crs.from_epsg(4326)

In [20]:
### set projection CRS WSG84 

gdf_stations.crs = {
                    'init' : 'epsg:%s' %(crs_WSG84),
                    'no_defs': True
                   }

print gdf_stations.crs


{'init': 'epsg:4326', 'no_defs': True}


In [21]:
### set indexes :

gdf_stations.set_index(["NUM_DEP", "NUM_COM", "NOM_COM",  "CD_ME_niv1_surf", "CD_ME_v2", "CD_STATION"], inplace=True) 
#gdf_stations.set_index("CD_STATION", inplace=True) 
gdf_stations.sort_index(inplace=True) 

print "-- gdf_stations.shape : ", gdf_stations.shape

-- gdf_stations.shape :  (13039, 20)


In [22]:
gdf_stations.head(7)

codagence  \
NUM_DEP NUM_COM NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
NaN     NaN     NaN               NaN             NaN      10261X0039/F3        AEAG   
                                                  CG004    01688X0034/AVAL      AERM   
                                                           01688X0039/F1        AERM   
01      01004   AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY      AERM&C   
        01006   AMBLEON           DG149           DG149    07007X0001/006A    AERM&C   
        01007   AMBRONAY          DG389           NaN      06754X0040/007A    AERM&C   
                                                  DG389    06754X0065/P2      AERM&C   

                                                                            ALTITUDE  \
NUM_DEP NUM_COM NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                  
NaN     NaN     NaN               NaN             NaN      10261X0039/F3        10.0   
                                                  CG004    01688X0034/AVAL     235.0   
                                                           01688X0039/F1       170.0   
01      01004   AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY       310.0   
        01006   AMBLEON           DG149           DG149    07007X0001/006A     420.0   
        01007   AMBRONAY          DG389           NaN      06754X0040/007A     243.0   
                                                  DG389    06754X0065/P2       243.0   

                                                                           PROFONDEUR_MAXI_POINT  \
NUM_DEP NUM_COM NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                              
NaN     NaN     NaN               NaN             NaN      10261X0039/F3                      33   
                                                  CG004    01688X0034/AVAL                   NaN   
                                                           01688X0039/F1                      80   
01      01004   AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY                     NaN   
        01006   AMBLEON           DG149           DG149    07007X0001/006A                   NaN   
        01007   AMBRONAY          DG389           NaN      06754X0040/007A                  12.5   
                                                  DG389    06754X0065/P2                      21   

                                                                                 reseau2009  \
NUM_DEP NUM_COM NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                         
NaN     NaN     NaN               NaN             NaN      10261X0039/F3    Hors RCS et RCO   
                                                  CG004    01688X0034/AVAL  Hors RCS et RCO   
                                                           01688X0039/F1    Hors RCS et RCO   
01      01004   AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY    Hors RCS et RCO   
        01006   AMBLEON           DG149           DG149    07007X0001/006A  Hors RCS et RCO   
        01007   AMBRONAY          DG389           NaN      06754X0040/007A  Hors RCS et RCO   
                                                  DG389    06754X0065/P2                RCS   

                                                                                 reseau2010  \
NUM_DEP NUM_COM NOM_COM           CD_ME_niv1_surf CD_ME_v2 CD_STATION                         
NaN     NaN     NaN               NaN             NaN      10261X0039/F3                NaN   
                                                  CG004    01688X0034/AVAL              NaN   
                                                           01688X0039/F1                NaN   
01      01004   AMBERIEU-EN-BUGEY DG149           DG149    06758X0052/HY    horsRCSRCODRIRE   
        01006   AMBLEON           DG149           DG149    07007X0001/006A              NaN   
        01007   AMBRONAY          DG389           NaN      06754X00

In [23]:
print list(gdf_stations.columns)

[u'codagence', u'ALTITUDE', u'PROFONDEUR_MAXI_POINT', u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', u'LAT_WSG84', u'LONG_WSG84', 'geometry']


In [24]:
### drop useless columns for geographic purposes

cols_to_drop = [
    #u'NUM_COM', 
    #u'codagence', 
    #u'ALTITUDE', 
    #u'PROFONDEUR_MAXI_POINT',
    u'reseau2009', u'reseau2010', u'reseau2011', u'reseau2012', u'reseau2013', u'reseau2014', 
    u'fi_ma_2007', u'fi_ma_2008', u'fi_ma_2009', u'fi_ma_2010', u'fi_ma_2011', u'fi_ma_2012', u'fi_ma_2013', u'fi_ma_2014', 
    #u'CD_STATION_', 
    #u'LAT_WSG84', 
    #u'LONG_WSG84', 
    #'geometry'
    ]
gdf_stations.drop(cols_to_drop, axis=1, inplace=True)


In [25]:
gdf_stations = gdf_stations.dropna( axis=0, how="all") # on empty rows


In [26]:
gdf_stations#.head()

codagence  \
NUM_DEP NUM_COM NOM_COM              CD_ME_niv1_surf CD_ME_v2 CD_STATION                    
NaN     NaN     NaN                  NaN             NaN      10261X0039/F3          AEAG   
                                                     CG004    01688X0034/AVAL        AERM   
                                                              01688X0039/F1          AERM   
01      01004   AMBERIEU-EN-BUGEY    DG149           DG149    06758X0052/HY        AERM&C   
        01006   AMBLEON              DG149           DG149    07007X0001/006A      AERM&C   
        01007   AMBRONAY             DG389           NaN      06754X0040/007A      AERM&C   
                                                     DG389    06754X0065/P2        AERM&C   
                                                              06754X0071/P00060    AERM&C   
        01008   AMBUTRIX             DG240           DG389    06758X0059/HY        AERM&C   
        01010   ANGLEFORT            DG148           DG148    07011X0028/HY        AERM&C   
                                     DG330           NaN      06775X0007/P2        AERM&C   
        01012   ARANC                DG149           DG149    06762X0022/P         AERM&C   
        01017   ARGIS                DG149           DG149    06766X0020/017A      AERM&C   
                                                              06766X0021/017B      AERM&C   
        01027   BALAN                DG326           NaN      06991D0172/F         AERM&C   
                                                     DG326    06991X0179/S2        AERM&C   
        01032   BELIGNEUX            DG390           DG390    06992X0035/S2        AERM&C   
                                     DG525           DG177    06991X0040/HY        AERM&C   
        01034   BELLEY               DG511           NaN      07008X0015/F2        AERM&C   
        01035   BELLEYDOUX           DG148           DG148    06531X0028/P00011    AERM&C   
        01036   BELMONT-LUTHEZIEU    DG511           DG511    07003X0017/036A      AERM&C   
        01041   BETTANT              DG240           DG149    06758X0055/HY        AERM&C   
        01043   BEYNOST              DG525           NaN      06984D0181/HY        AERM&C   
        01044   BILLIAT              DG511           DG511    06771X0012/HY        AERM&C   
        01049   LA BOISSE            DG326           DG177    06984D0286/P00026    AERM&C   
        01051   BOLOZON              DG149           DG140    06521X0016/HY        AERM&C   
        01053   BOURG-EN-BRESSE      DG342           DG342    06513X0028/F         AERM&C   
        01061   BRENS                DG326           DG326    07244X0030/P         AERM&C   
        01064   BRIORD               DG326           NaN      07005X0133/F         AERM&C   
        01066   LA BURBANCHE         DG149           DG149    07002X0008/HY        AERM&C   
...                                                                                   ...   
95      95355   MAGNY-EN-VEXIN       HG107           NaN      01265X1006/HY          AESN   
                                                              01265X1019/HY          AESN   
        95392   MERIEL               HG002           NaN      01531X0031/P3          AESN   
                                                              01531X0093/P           AESN   
                                                     HG002    01531X0096/P5          AESN   
        95409   MOISSELLES           HG104           NaN      01536X0159/F           AESN   
        95422   MONTGEROULT          HG107           NaN      01523X0031/F           AESN   
                                                              01523X0032/F           AESN   
                                                              01523X0033/S           AESN   
        95430   MONTSOULT            HG104           NaN      01532X0047/F           AESN   
        95436   MOURS                HG002           NaN      01531X0087/CASAN2  

In [27]:
#gdf_stations = gdf_stations.reset_index()
#gdf_stations.head(3)

In [28]:
### save geojson for stations


### overule Fiona overwrite bug
try: 
    os.remove(out_filepath)
except OSError:
    pass

### filepath for stations_web.geojson
out_filename = "stations_web_carto.geojson"
out_filepath = os.path.join(carto_web_path, out_filename )


#fiona.supported_drivers
gdf_stations.to_file( out_filepath, driver="GeoJSON" )

